# Graph Convolution Network (GCN) for a classification problem

We will work with the CORA dataset from pytorch geometirc.
Each paper is assingned a high dimensional vector based on its contents
and we will work with the directed graph formed by the citations.
Each paper or node is classified based on research topic.


In [15]:
from torch_geometric.datasets import Planetoid

# Load the CORA dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')

# Access the first graph object
data = dataset[0]


The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [28]:
print("No. of nodes = ", data['x'].shape[0])
print("No. of edges = ", data['edge_index'].shape[1])
print("Classification labels range from ", data['y'].min(), " to ", data['y'].max())

No. of nodes =  2708
No. of edges =  10556
Classification labels range from  tensor(0)  to  tensor(6)
